In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import itertools
import cv2
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report

In [122]:
df = pd.read_csv('GroundTruth.csv')
df.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
df.shape

(10015, 8)

In [124]:
dfNV = df[df["NV"]==1]

In [125]:
dfNV.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,ISIC_0024311,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [126]:
dfNV.drop(columns=["MEL", "BCC", "AKIEC", "BKL", "DF", "VASC"], inplace=True)

<ipython-input-126-307c0a01f0b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNV.drop(columns=["MEL", "BCC", "AKIEC", "BKL", "DF", "VASC"], inplace=True)


In [127]:
dfNV.head()

,image,NV
0,ISIC_0024306,1.0
1,ISIC_0024307,1.0
2,ISIC_0024308,1.0
3,ISIC_0024309,1.0
5,ISIC_0024311,1.0


In [128]:
dfNotNV = df[df["NV"]==0]

In [129]:
dfNotNV.tail()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
10007,ISIC_0034313,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10009,ISIC_0034315,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10010,ISIC_0034316,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10011,ISIC_0034317,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10012,ISIC_0034318,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [130]:
dfNotNV.drop(columns=["MEL", "BCC", "AKIEC", "BKL", "DF", "VASC"], inplace=True)

<ipython-input-130-ea989bb29842>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNotNV.drop(columns=["MEL", "BCC", "AKIEC", "BKL", "DF", "VASC"], inplace=True)


In [131]:
dfNotNV.head()

,image,NV
4,ISIC_0024310,0.0
6,ISIC_0024312,0.0
7,ISIC_0024313,0.0
9,ISIC_0024315,0.0
12,ISIC_0024318,0.0


In [132]:
dfNotNV.shape

(3310, 2)

In [133]:
dfNV.shape

(6705, 2)

In [134]:
dfNV = dfNV.sample(frac = 0.5, random_state = 40)

In [135]:
dfNV.shape

(3352, 2)

In [136]:
dfNotNV.shape

(3310, 2)

In [137]:
df = pd.concat([dfNV, dfNotNV])

In [138]:
df.head()

,image,NV
4099,ISIC_0028405,1.0
840,ISIC_0025146,1.0
7442,ISIC_0031748,1.0
2880,ISIC_0027186,1.0
6001,ISIC_0030307,1.0


In [139]:
df.shape

(6662, 2)

In [140]:
df = df.sample(frac=1, random_state=30)

In [141]:
df.head()

,image,NV
7638,ISIC_0031944,1.0
5034,ISIC_0029340,0.0
1405,ISIC_0025711,0.0
7882,ISIC_0032188,1.0
128,ISIC_0024434,1.0


In [142]:
imageArray = df["image"].to_numpy()

In [143]:
imageArray.shape

(6662,)

In [144]:
pwd

'/workspace/aman/MANYA'

In [145]:
cd images

/workspace/aman/MANYA/images


In [146]:
pwd

'/workspace/aman/MANYA/images'

In [147]:
Images = []

In [148]:
def processImage(image):
    image = cv2.resize(image, (150, 200))
    image = image.astype("float32") / 255
    return image

In [149]:
for img in imageArray:
    path = img + ".jpg"
    image = cv2.imread(path)
    image = processImage(image)
    Images.append(image)

In [150]:
len(Images)

6662

In [155]:
cd ..

/workspace/aman/MANYA


In [156]:
np.save("NVIMAGES.npy", Images)

In [157]:
output = df["NV"].to_numpy()

In [158]:
len(output)

6662

In [159]:
output.shape

(6662,)

In [160]:
xTrain, xTest, yTrain, yTest = train_test_split(np.array(Images), output, test_size = 0.2, random_state = 30)

In [161]:
xTrain.shape

(5329, 200, 150, 3)

In [162]:
yTrain.shape

(5329,)

In [163]:
xTrain, xVal, yTrain, yVal = train_test_split(xTrain, yTrain, test_size = 0.2, random_state = 30)

In [164]:
xVal.shape

(1066, 200, 150, 3)

In [165]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=(200, 150, 3)))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 200, 150, 32)      896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 200, 150, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 100, 75, 32)       0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 75, 32)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 100, 75, 64)       18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 100, 75, 64)       36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 50, 37, 64)       

In [166]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["Accuracy"])

In [167]:
history = model.fit(xTrain, yTrain, validation_data = (xVal, yVal), epochs = 10, batch_size = 32)

Epoch 1/10
134/134 [==============================] - 5s 34ms/step - loss: 1.7780 - accuracy: 0.0187 - val_loss: 0.5994 - val_accuracy: 0.0000e+00
Epoch 2/10
134/134 [==============================] - 4s 30ms/step - loss: 0.5608 - accuracy: 0.0000e+00 - val_loss: 0.5312 - val_accuracy: 0.0000e+00
Epoch 3/10
134/134 [==============================] - 4s 30ms/step - loss: 0.5747 - accuracy: 0.0000e+00 - val_loss: 0.6356 - val_accuracy: 0.0000e+00
Epoch 4/10
134/134 [==============================] - 4s 30ms/step - loss: 0.5498 - accuracy: 0.0000e+00 - val_loss: 0.4944 - val_accuracy: 0.0000e+00
Epoch 5/10
134/134 [==============================] - 4s 29ms/step - loss: 0.5109 - accuracy: 0.0000e+00 - val_loss: 0.5519 - val_accuracy: 0.0000e+00
Epoch 6/10
134/134 [==============================] - 4s 29ms/step - loss: 0.5295 - accuracy: 0.0000e+00 - val_loss: 0.4768 - val_accuracy: 0.0000e+00
Epoch 7/10
134/134 [==============================] - 4s 29ms/step - loss: 0.4891 - accuracy: 0.00

In [168]:
predictions = model.predict(xTest)

In [169]:
preds = []

In [170]:
for p in predictions:
    if p>=0.5:
        preds.append(1)
    else:
        preds.append(0)

In [171]:
report = classification_report(yTest, preds, target_names = ["NOT NV", "NV"])

In [172]:
print(report)

              precision    recall  f1-score   support

      NOT NV       0.74      0.82      0.78       652
          NV       0.81      0.72      0.76       681

    accuracy                           0.77      1333
   macro avg       0.77      0.77      0.77      1333
weighted avg       0.78      0.77      0.77      1333

